<a href="https://colab.research.google.com/github/lucaskawazoi/lkk-xview2/blob/master/localization/create_model_fn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

References:
- https://cloud.google.com/tpu/docs/tutorials/mnist#download_and_convert_the_mnist_data
- /home/lucaskawazoi/development/professional/temp/tpu/models/official/mnist
- /home/lucaskawazoi/development/professional/lkk-xview2/localization
- Continuation of lkk-xview2/localization/create_tfrecords.ipynb

# Import libraries

In [1]:
# Import libraries
import os
from pathlib import Path

import numpy as np 
import math
import random

import cv2
from google.colab.patches import cv2_imshow

import tensorflow as tf
print('TF version:', tf.__version__)

TF version: 1.15.0


# Authenticate and set variables

In [4]:
# Authenticate and config credentials
! gcloud auth login
! gcloud config set project lkk-project-1

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?code_challenge=dlm6KiOXdb8YkWvaocA7l-c-F1rQX5HtfH_DZwP_NSk&prompt=select_account&code_challenge_method=S256&access_type=offline&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&client_id=32555940559.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth


Enter verification code: 4/uAFXqGuK9bQqt0ScaLuJKAL2JC2_i9fNb5U87vHgvzqhlaJwTNlaP_E
If you need to use ADC, see:
  gcloud auth application-default --help

You are now logged in as [lucas.key.kawazoi@gmail.com].
Your current project is [None].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID
Updated property [core/project].


In [0]:
# Set folder paths
PATH = Path()
images_folder = PATH / 'images'
images_folder.mkdir(exist_ok=True)

grayscale_labels_folder = PATH / 'grayscale_labels'
grayscale_labels_folder.mkdir(exist_ok=True)

dataSet_folder = PATH / 'dataSet'
dataSet_folder.mkdir(exist_ok=True)

tfrecords_folder = PATH / 'tfrecords'
tfrecords_folder.mkdir(exist_ok=True)

model_dir = Path()

dataset = 'train'

# Create dataset_fn

In [6]:
# Download data
! gsutil -m rsync -r gs://lkk-xview2/xBD/spacenet_gt/tfrecords {tfrecords_folder}

Building synchronization state...
Starting synchronization...
Copying gs://lkk-xview2/xBD/spacenet_gt/tfrecords/train-00000-of-00004.tfrecord...
Copying gs://lkk-xview2/xBD/spacenet_gt/tfrecords/train-00002-of-00004.tfrecord...
Copying gs://lkk-xview2/xBD/spacenet_gt/tfrecords/val-00002-of-00004.tfrecord...
Copying gs://lkk-xview2/xBD/spacenet_gt/tfrecords/train-00003-of-00004.tfrecord...
Copying gs://lkk-xview2/xBD/spacenet_gt/tfrecords/val-00003-of-00004.tfrecord...
Copying gs://lkk-xview2/xBD/spacenet_gt/tfrecords/train-00001-of-00004.tfrecord...
Copying gs://lkk-xview2/xBD/spacenet_gt/tfrecords/val-00000-of-00004.tfrecord...
Copying gs://lkk-xview2/xBD/spacenet_gt/tfrecords/val-00001-of-00004.tfrecord...
/ [8/8 files][  3.8 GiB/  3.8 GiB] 100% Done  28.0 MiB/s ETA 00:00:00           
Operation completed over 8 objects/3.8 GiB.                                      


In [0]:
# Get desired filenames
tfrecords_filenames = sorted(tfrecords_folder.rglob('**/{}-*.tfrecord'.format(dataset)))
tfrecords_filenames = [str(f) for f in tfrecords_filenames]

def create_dataset(tfrecords_filenames):
  feature_description = {
    'image_encoded': tf.io.FixedLenFeature([], tf.string),
    'image_filename': tf.io.FixedLenFeature([], tf.string),
    'height': tf.io.FixedLenFeature([], tf.int64),
    'width': tf.io.FixedLenFeature([], tf.int64),
    'depth': tf.io.FixedLenFeature([], tf.int64),
    'label_encoded': tf.io.FixedLenFeature([], tf.string),
    'label_filename': tf.io.FixedLenFeature([], tf.string),
    }
  def decode_record(record):
    example = tf.io.parse_single_example(record, feature_description)
    
    # image_nparray = np.frombuffer(example['image_encoded'], np.uint8)
    # image = cv2.imdecode(image_nparray, cv2.IMREAD_COLOR)
    image = tf.decode_raw(example['image_encoded'], tf.uint8)
    image = tf.cast(image, tf.float32)
    image = tf.reshape(image, [1024, 1024, 3])

    # label_nparray = np.frombuffer(example['label_encoded'], np.uint8)
    # label = cv2.imdecode(label_nparray, cv2.IMREAD_COLOR)
    label = tf.decode_raw(example['label_encoded'], tf.uint8)
    label = tf.cast(label, tf.float32)
    label = tf.reshape(label, [1024, 1024, 1])
    return image, label

  return tf.data.TFRecordDataset(tfrecords_filenames).map(decode_record)

def train_input_fn(batch_size, tfrecords_filenames):
  """train_input_fn defines the input pipeline used for training."""
  # batch_size = params['batch_size']
  # tfrecords_filenames = params['tfrecords_folder'] / "train.tfrecords")

  return create_dataset(tfrecords_filenames).cache().repeat().shuffle(
      buffer_size=4000).batch(batch_size, drop_remainder=True)

# Connect to TPU

In [0]:
# TPUClusterResolver
tpu_cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver()

In [0]:
# tpu.RunConfig
iterations = 50
num_shards = 8
run_config = tf.estimator.tpu.RunConfig(
      cluster=tpu_cluster_resolver,
      model_dir=model_dir,
      session_config=tf.ConfigProto(
          allow_soft_placement=True, log_device_placement=True),
      tpu_config=tf.estimator.tpu.TPUConfig(iterations, num_shards),
  )

# model_fn

In [0]:
def _unet_model_fn(image, labels, mode, params):
  with tf.variable_scope('base', reuse=tf.AUTO_REUSE):
    logits = unet3d_base(
          image,
          pool_size=(2, 2, 2),
          n_labels=params['num_classes'],
          deconvolution=params['deconvolution'],
          depth=params['depth'],
          n_base_filters=params['num_base_filters'],
          batch_normalization=params['use_batch_norm'],
          data_format=params['data_format'])

In [0]:
def unet_model_fn(features, labels, mode, params):
  """UNet model."""
  with tf.variable_scope('unet', reuse=tf.AUTO_REUSE):
    return _unet_model_fn(features, labels, mode, params)

In [0]:
def model_fn(features, labels, mode, params):

  del params

  # Normalize from [0, 255] to [0.0, 1.0]
  image = features / 255.

  y = tf.layers.Conv2D(filters=32, kernel_size=5, padding='same', activation='relu')(image)
  y = tf.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="same")(y)
  y = tf.layers.Conv2D(filters=32, kernel_size=5, padding="same", activation="relu")(y)
  y = tf.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="same")(y)
  y = tf.layers.Flatten()(y)
  y = tf.layers.Dense(1024, activation="relu")(y)
  y = tf.layers.Dropout(0.4)(y, training=(mode == tf.estimator.ModeKeys.TRAIN))

  logits = tf.layers.Dense(10)(y)

# tpu.TPUEstimator

In [0]:
estimator = tf.estimator.tpu.TPUEstimator(
      model_fn=model_fn,
      use_tpu=FLAGS.use_tpu,
      train_batch_size=FLAGS.batch_size,
      eval_batch_size=FLAGS.batch_size,
      predict_batch_size=FLAGS.batch_size,
      params={"data_dir": FLAGS.data_dir},
      config=run_config)

# The end